In [0]:
!git clone https://github.com/kak-to-tak/Google_rusngram_spellcheck

Cloning into 'Google_rusngram_spellcheck'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 99 (delta 37), reused 46 (delta 13), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [0]:
!pip3 install -r Google_rusngram_spellcheck/requirements.txt

In [0]:
%cd Google_rusngram_spellcheck/main/

/content/Google_rusngram_spellcheck/main


In [0]:
!sh ../data/download.sh

In [0]:
from SpellCorrect import spellCorrect
import pandas as pd

In [0]:
class Evaluation(object):
    def __init__(self):
        self.correct = spellCorrect()
        self.val_df = pd.read_csv('val_df_lm.csv')
        self.tp, self.fp, self.tn, self.fn = [], [], [],[]
        self.prec, self.rec, self.f1 = [],[],[]
        self.acc = []
        self.corrections = self.correct_to_check()
        self.boundaries = [[0,0]]
    
    def define_rates(self):
        try:
            tp = self.val_df[(self.val_df['rough_corrections']==self.val_df['new_ngram']) \
                                 & (self.val_df['new_ngram']!=self.val_df['n_gram'])]
            self.tp.append(tp)
            
            fp =  self.val_df[(self.val_df['rough_corrections']!=self.val_df['new_ngram']) \
                                  & (self.val_df['new_ngram']==self.val_df['n_gram'])]
            self.fp.append(fp)
            
            fn = self.val_df[(self.val_df['rough_corrections']==self.val_df['n_gram']) & \
                        (self.val_df['new_ngram']!=self.val_df['n_gram']) & \
                 (self.val_df['decision']!='транслитерация') \
                 & (self.val_df['decision']!='меньше пяти символов')]
            self.fn.append(fn)
            
            tn = self.val_df[((self.val_df['rough_corrections']==self.val_df['n_gram']) \
                                 & (self.val_df['new_ngram']==self.val_df['n_gram'])) \
                             |  ((self.val_df['rough_corrections']==self.val_df['n_gram']) & \
                        (self.val_df['decision']=='транслитерация'))]
            self.tn.append(tn)
        
        except KeyError:
            print ("No corrections to evaluate\nAt first use SpellCorrect")
            return [],[],[]
    
    def evaluate(self):
        if (len(self.tp[-1])+len(self.fp[-1])) == 0:
            self.prec.append(0)
        else:
            self.prec.append(len(self.tp[-1]) / (len(self.tp[-1]) + len(self.fp[-1])))

        if (len(self.tp[-1])+len(self.fn[-1])) == 0:
            self.rec.append(0)
        else:
            self.rec.append(len(self.tp[-1]) / (len(self.tp[-1]) + len(self.fn[-1])))
        if (self.prec[-1]+self.rec[-1]) == 0:
            self.f1.append(0)
        else:
            self.f1.append((2*(self.prec[-1]*self.rec[-1]))/(self.prec[-1]+self.rec[-1]))
            
        if (len(self.tp[-1])+len(self.tn[-1])+len(self.fn[-1])+len(self.fp[-1]))==0: 
            self.acc.append(0)
        else:
            self.acc.append((len(self.tp[-1])+len(self.tn[-1]))/(len(self.tp[-1])+len(self.tn[-1])+len(self.fn[-1])+len(self.fp[-1])))
            
        if self.prec[-1]>0.5:
          print('Precision - ', round(self.prec[-1], 2))
          print('Recall - ', round(self.rec[-1], 2))
          print('F1 - ', round(self.f1[-1], 2))
          print('Accuracy - ', round(self.acc[-1], 2))
          print("="*60)
        
    def myCoolGridSearch(self, rules=False):
        "Because I didn't know how to use that one here"
        model_probs = self.val_df['model_prob']
        params = sorted(list(set(model_probs)))
        for upper_boundary in params[::-1]:
            for lower_boundary in params:
              self.boundaries.append([upper_boundary, lower_boundary])
              
              self.evaluate_with_boudary(upper_boundary, lower_boundary, rules)
              if self.prec[-1]>0.5:
                print('Upper boundary = {}\nLower boundary = {}'.format(upper_boundary,lower_boundary))
              if upper_boundary<lower_boundary:
                break
              
    def correct_to_check(self):
      model_probs = self.val_df['model_prob']
      words = self.val_df['n_gram']
      corrections=[self.correct.to_check(word, model_probs.iloc[[i]][i], 0, 0) for i, word in enumerate(words)]
      self.val_df['rough_corrections'] = corrections
      self.define_rates()
      self.evaluate()
      
      return corrections
    
    def evaluate_with_boudary(self, upper, lower, rules=False):
      words = self.val_df['n_gram']
      model_probs = self.val_df['model_prob']
      if upper!=0 and lower!=0:
        if rules:
          new_corrections=[correction if model_probs[[i]][i] < upper and model_probs[[i]][i] > lower \
                             else words[[i]][i] if not self.with_rules(words[[i]][i], correction) \
                             else correction for i, correction in enumerate(self.corrections)]
          
        else:
          new_corrections=[correction if model_probs[[i]][i] < upper and model_probs[[i]][i] > lower \
                                 else words[[i]][i] for i, correction in enumerate(self.corrections)]
        self.val_df['rough_corrections'] = new_corrections
      else:
        self.val_df['rough_corrections'] = self.corrections  
      self.define_rates()
      self.evaluate()
      
    def with_rules(self, word, correction):
      if len(word)>4:
        rule1 = word[-1] in 'шщцжЦШЩЖ' and len(word)>5
        rule2 = any(i.isdigit() for i in word)
        rule3 = word[-2] in 'щшцЩШЦ' and word[-1] in 'июяИЮЯ' and len(word)>5
        rule4 = ('ѣ' or 'Ѣ')  in correction
        if rule1 or rule2 or rule3 or rule4:
          return True
      return False

In [0]:
%%time
evaluation = Evaluation()

Loading dictionary from data file
Loading counts
Loading error dataframe
Precision -  0.24
Recall -  0.91
F1 -  0.38
Accuracy -  0.52
CPU times: user 24min 29s, sys: 12.4 s, total: 24min 42s
Wall time: 24min 42s


In [0]:
%%time
evaluation.myCoolGridSearch()

In [0]:
evaluation.evaluate_with_boudary(0.11082839220762253, 0.09979256987571716)

Precision -  0.71
Recall -  0.04
F1 -  0.07
Accuracy -  0.75


In [0]:
evaluation.tp[-1]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
129,Аруга_NOUN,1778,1,1,a,Аруга,NaN,NaN,Друга,исправлено,0.108465,Друга
193,аанимает,1899,1,1,a,аанимает,NaN,NaN,занимает,исправлено,0.107925,занимает
223,адоровыхъ,1831,1,1,a,адоровыхъ,NaN,NaN,здоровыхъ,исправлено,0.099915,здоровыхъ
370,ядляется_VERB,1896,1,1,a,ядляется,NaN,NaN,является,исправлено,0.110328,является
383,яолучить_VERB,1885,1,1,a,яолучить,NaN,NaN,получить,исправлено,0.109294,получить


In [0]:
evaluation.fp[-1]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
250,алемановъ,1869,5,3,a,алемановъ,NaN,0.0,алемановъ,не нуждается в исправлении,0.099940,алеманов
456,Албрехтъ_NOUN,1811,1,1,a,Албрехтъ,NaN,0.0,Албрехтъ,не нуждается в исправлении,0.109629,Альбрехтъ


In [0]:
evaluation.evaluate_with_boudary(0.11082839220762253, 0.0999404564499855)

Precision -  0.8
Recall -  0.03
F1 -  0.06
Accuracy -  0.75


In [0]:
evaluation.tp[-1]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
129,Аруга_NOUN,1778,1,1,a,Аруга,NaN,NaN,Друга,исправлено,0.108465,Друга
193,аанимает,1899,1,1,a,аанимает,NaN,NaN,занимает,исправлено,0.107925,занимает
370,ядляется_VERB,1896,1,1,a,ядляется,NaN,NaN,является,исправлено,0.110328,является
383,яолучить_VERB,1885,1,1,a,яолучить,NaN,NaN,получить,исправлено,0.109294,получить


In [0]:
evaluation.fp[-1]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
456,Албрехтъ_NOUN,1811,1,1,a,Албрехтъ,NaN,0.0,Албрехтъ,не нуждается в исправлении,0.109629,Альбрехтъ


In [0]:
evaluation.evaluate_with_boudary(0.109628863632679, 0.09979256987571716)

Precision -  0.8
Recall -  0.03
F1 -  0.06
Accuracy -  0.75


In [0]:
evaluation.tp[-1]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
203,авсгро,1859,1,1,a,авсгро,NaN,NaN,австро,исправлено,0.000329,австро
282,анллиза,1887,1,1,a,анллиза,NaN,1.0,анализа,исправлено,0.000860,анализа
306,апрьля,1836,6,2,a,апрьля,NaN,NaN,апрѣля,исправлено,0.000553,апрѣля
325,архиеписко_ADJ,1841,1,1,a,архиеписко,NaN,NaN,архиепископ,исправлено,0.000531,архиепископ


In [0]:
evaluation.fp[-1]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
119,Аргишти_NOUN,1880,26,1,a,Аргишти,NaN,0.0,Аргишти,не нуждается в исправлении,0.000877,Артисти


### After applying the rules

In [0]:
%%time
evaluation.myCoolGridSearch(rules=True)

In [0]:
# for idx, rec in enumerate(evaluation.rec):
#   if rec > 0.11:
#     if evaluation.prec[idx]>0.72:
#       print('recall= {}\nidx = {}\nprecicion = {}\nf1 = {}\naccuracy = {}\nboundaries={}'.format(rec, 
#                                                                                                  idx,
#                                                                                                  evaluation.prec[idx], 
#                                                                                                  evaluation.f1[idx], 
#                                                                                                  evaluation.acc[idx], 
#                                                                                                  evaluation.boundaries
#                                                                                                  [idx]))
#       print('='*50)

In [0]:
evaluation.boundaries[41749]

[0.0008771931170485914, 0.0003082964103668928]

In [0]:
evaluation.tp[41749]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
72,Александровича2_NOUN,1869,2,2,a,Александровича2,NaN,1.0,Александровича,исправлено,4.251733e-26,Александровича
177,Японск1е_NOUN,1786,1,1,a,Японск1е,NaN,NaN,Японскіе,исправлено,7.789776e-14,Японскіе
203,авсгро,1859,1,1,a,авсгро,NaN,NaN,австро,исправлено,3.292023e-04,австро
235,академическ1я,1818,1,1,a,академическ1я,NaN,NaN,академическія,исправлено,2.189020e-10,академическія
282,анллиза,1887,1,1,a,анллиза,NaN,1.0,анализа,исправлено,8.601645e-04,анализа
306,апрьля,1836,6,2,a,апрьля,NaN,NaN,апрѣля,исправлено,5.533596e-04,апрѣля
315,армянск1й_ADJ,1860,1,1,a,армянск1й,NaN,NaN,армянскій,исправлено,4.826314e-08,армянскій
325,архиеписко_ADJ,1841,1,1,a,архиеписко,NaN,NaN,архиепископ,исправлено,5.305539e-04,архиепископ
342,ассигнован1я,1830,1,1,a,ассигнован1я,NaN,NaN,ассигнованія,исправлено,4.298733e-09,ассигнованія
345,астроном1И,1821,1,1,a,астроном1И,NaN,NaN,астрономіи,исправлено,9.900248e-14,астрономіи


In [0]:
evaluation.fp[41749]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
120,Ардеш,1852,1,1,a,Ардеш,NaN,0.0,Ардеш,не нуждается в исправлении,1.445414e-07,Орден
257,альн1_NOUN,1889,1,1,a,альн1,NaN,NaN,альн1,не получилось принять решение,5.517670e-09,альны
397,ясненш_NOUN,1786,1,1,a,ясненш,NaN,NaN,ясненш,не получилось принять решение,1.064793e-11,ясенъ


In [0]:
evaluation.boundaries[12534]

[0.11082839220762253, 0.09979256987571716]

In [0]:
evaluation.tp[12534]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
72,Александровича2_NOUN,1869,2,2,a,Александровича2,NaN,1.0,Александровича,исправлено,4.251733e-26,Александровича
129,Аруга_NOUN,1778,1,1,a,Аруга,NaN,NaN,Друга,исправлено,1.084646e-01,Друга
177,Японск1е_NOUN,1786,1,1,a,Японск1е,NaN,NaN,Японскіе,исправлено,7.789776e-14,Японскіе
193,аанимает,1899,1,1,a,аанимает,NaN,NaN,занимает,исправлено,1.079250e-01,занимает
223,адоровыхъ,1831,1,1,a,адоровыхъ,NaN,NaN,здоровыхъ,исправлено,9.991474e-02,здоровыхъ
235,академическ1я,1818,1,1,a,академическ1я,NaN,NaN,академическія,исправлено,2.189020e-10,академическія
315,армянск1й_ADJ,1860,1,1,a,армянск1й,NaN,NaN,армянскій,исправлено,4.826314e-08,армянскій
342,ассигнован1я,1830,1,1,a,ассигнован1я,NaN,NaN,ассигнованія,исправлено,4.298733e-09,ассигнованія
345,астроном1И,1821,1,1,a,астроном1И,NaN,NaN,астрономіи,исправлено,9.900248e-14,астрономіи
370,ядляется_VERB,1896,1,1,a,ядляется,NaN,NaN,является,исправлено,1.103282e-01,является


In [0]:
evaluation.fp[12534]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
120,Ардеш,1852,1,1,a,Ардеш,NaN,0.0,Ардеш,не нуждается в исправлении,1.445414e-07,Орден
250,алемановъ,1869,5,3,a,алемановъ,NaN,0.0,алемановъ,не нуждается в исправлении,9.994046e-02,алеманов
257,альн1_NOUN,1889,1,1,a,альн1,NaN,NaN,альн1,не получилось принять решение,5.517670e-09,альны
397,ясненш_NOUN,1786,1,1,a,ясненш,NaN,NaN,ясненш,не получилось принять решение,1.064793e-11,ясенъ
456,Албрехтъ_NOUN,1811,1,1,a,Албрехтъ,NaN,0.0,Албрехтъ,не нуждается в исправлении,1.096289e-01,Альбрехтъ


In [0]:
evaluation.evaluate_with_boudary(0.0008771931170485914, 0.0003082964103668928, rules=True)

Precision -  0.74
Recall -  0.13
F1 -  0.23
Accuracy -  0.8


In [0]:
evaluation.fp[-1]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
120,Ардеш,1852,1,1,a,Ардеш,NaN,0.0,Ардеш,не нуждается в исправлении,1.445414e-07,Орден
180,Яроши_NOUN,1875,1,1,a,Яроши,NaN,0.0,Яроши,не нуждается в исправлении,5.950703e-05,Гроши
257,альн1_NOUN,1889,1,1,a,альн1,NaN,NaN,альн1,не получилось принять решение,5.517670e-09,альны
397,ясненш_NOUN,1786,1,1,a,ясненш,NaN,NaN,ясненш,не получилось принять решение,1.064793e-11,ясенъ
440,Адаше,1833,1,1,a,Адаше,NaN,0.0,Адаше,не нуждается в исправлении,1.416731e-03,Бѣаше


In [0]:
evaluation.tp[-1]

,raw_ngram,year,match_count,volume_count,idx,n_gram,new_idx,is_bastard,new_ngram,decision,model_prob,rough_corrections
69,АлексЪй_NOUN,1785,3,1,a,АлексЪй,NaN,1.0,Алексѣй,исправлено,1.001208e-10,Алексѣй
72,Александровича2_NOUN,1869,2,2,a,Александровича2,NaN,1.0,Александровича,исправлено,4.251733e-26,Александровича
177,Японск1е_NOUN,1786,1,1,a,Японск1е,NaN,NaN,Японскіе,исправлено,7.789776e-14,Японскіе
203,авсгро,1859,1,1,a,авсгро,NaN,NaN,австро,исправлено,3.292023e-04,австро
235,академическ1я,1818,1,1,a,академическ1я,NaN,NaN,академическія,исправлено,2.189020e-10,академическія
282,анллиза,1887,1,1,a,анллиза,NaN,1.0,анализа,исправлено,8.601645e-04,анализа
306,апрьля,1836,6,2,a,апрьля,NaN,NaN,апрѣля,исправлено,5.533596e-04,апрѣля
315,армянск1й_ADJ,1860,1,1,a,армянск1й,NaN,NaN,армянскій,исправлено,4.826314e-08,армянскій
325,архиеписко_ADJ,1841,1,1,a,архиеписко,NaN,NaN,архиепископ,исправлено,5.305539e-04,архиепископ
342,ассигнован1я,1830,1,1,a,ассигнован1я,NaN,NaN,ассигнованія,исправлено,4.298733e-09,ассигнованія
